# Class 9B: Interactions I

## Learning Objectives
By completing this workbook, you will be able to:
- Explain the parallel structure between Altair's visualization grammar and interaction grammar
- Implement point selections with three different interaction patterns (click, hover, nearest)
- Implement interval selections with axis constraints (both axes, x-only, y-only)
- Apply `alt.when().then().otherwise()` conditional logic to create visual feedback for selections
- Create coordinated multi-view visualizations using `transform_filter()` to link selection in one chart to filtering in another
- Choose appropriate selection types and patterns based on analytical goals and data characteristics

---

## Section A: Conceptual Foundation

### The Parallel Declarative Grammars

Both visualization and interaction in Altair use **declarative grammars** - you specify *what* you want, not *how* to achieve it. Understanding this parallel will help you master interactive visualizations.

#### Visualization Grammar → Interaction Grammar

| **Visualization Grammar** | **Interaction Grammar** | **Mental Model** |
|---|---|---|
| **Data Attributes** | **Parameter Values** | The "raw material" you reference |
| **Encoding Channels** | **Conditions** | How you map to visual properties |
| **Marks** (prop r static (e.g. size, opacity) | **Marks** (with dynamic properties) | What the user sees |

---

### Core Insight: Static vs. Dynamic Mappings

Both grammars declaratively map values to visual properties. The difference is **where the values come from**.
#### **Visualization: Mapping DATA to Visuals**

In [ ]:
import altair as alt
from vega_datasets import data

cars = data.cars()

# Static visualization
alt.Chart(cars).mark_circle().encode(
    x='Horsepower:Q',       # Data attribute → x position
    y='Miles_per_Gallon:Q', # Data attribute → y position
    color='Origin:N'        # Data attribute → color
)




**Flow:** `Data (cars.csv) → encode in channels → render as marks → Visual`

---

#### **Interaction: Mapping PARAMETERS to Visuals**

**Example 1: Variable Parameter (User-controlled constant)**


In [ ]:
# Create a parameter that stores a number
opacity_param = alt.param(
    value=0.7,  # Initial value
    bind=alt.binding_range(min=0.1, max=1.0, step=0.1, name='Opacity')
)

# Dynamic visualization
alt.Chart(cars).mark_circle().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    opacity=opacity_param  # Parameter value → opacity
).add_params(
    opacity_param
)


**Flow:** `User moves slider → parameter updates → opacity changes → Visual updates`

---

**Example 2: Selection Parameter (User-controlled data query)**


In [ ]:
# Create a parameter that captures user brushing
brush = alt.selection_interval()

# Dynamic visualization with conditional encoding
alt.Chart(cars).mark_circle().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    opacity=alt.when(brush).then(alt.value(1.0)).otherwise(alt.value(0.2))  # Parameter state → opacity
).add_params(
    brush
)


**Flow:** `User brushes → selection captures points → condition evaluates → opacity changes → Visual updates`


### The Pattern: Static → Dynamic

The key insight is that **any static value can become dynamic** by replacing it with a parameter:

| **Static (Visualization)** | **Dynamic (Interaction)** |
|---|---|
| `opacity=0.7` | `opacity=opacity_param` |
| `color='Origin:N'` | `color=alt.when(brush).then('Origin:N').otherwise(alt.value('gray'))` |
| `size=60` | `size=alt.when(selection).then(alt.value(100)).otherwise(alt.value(60))` |

---

### The Philosophy of Altair v5 Interactions

**Parameters are the building blocks of all interactions in Altair v5.** This fundamental concept shapes how we think about making visualizations interactive. Rather than thinking about "adding interactivity" to a static chart, we design with parameters from the ground up.

#### Two Types of Parameters

Altair v5 uses two distinct types of parameters, each serving different purposes:

**1. Variable Parameters:** Store single values (numbers, strings, booleans)
- Created with `alt.param(value=...)`
- Reference directly: `opacity=opacity_param`
- Think of variable parameters as "smart constants" - they hold values that can be updated through user interface controls, but they don't directly capture mouse events or user gestures. They are bound to widgets: sliders, dropdowns, checkboxes

**2. Selection Parameters:** Store data queries (which points/intervals are selected)
- Created with `alt.selection_point()` or `alt.selection_interval()`
- Test with conditions: `alt.when(selection).then(...).otherwise(...)`
- Are designed to capture user interactions with the visualization itself - clicking on points, brushing over regions, or hovering over elements.

#### The Parameter Lifecycle

Every interactive visualization follows this lifecycle:

```
1. Define → 2. Register → 3. Reference → 4. React
```

1. **Define**: Create the parameter with its type and configuration
   - Variable: `alt.param(value=0.7, bind=alt.binding_range(...))`
   - Selection: `alt.selection_interval()`

2. **Register**: Add it to your chart so it can respond to user input
   - `alt.Chart(data).add_params(opacity_param)`
   - `alt.Chart(data).add_params(brush)`

3. **Reference**: Use the parameter in your chart specification
   - Variable: `opacity=opacity_param`
   - Selection: `opacity=alt.when(brush).then(alt.value(1)).otherwise(alt.value(0.2))`

4. **React**: The visualization automatically updates when the parameter changes
   - Happens behind the scenes in the browser

---

#### Interaction Flow

The basic flow of interaction in Altair follows this pattern:

```
User Action → Parameter Update → Visual Encoding Change
```

**Example with Variable Parameter:**
```
User moves slider → opacity_param value changes → circle opacity updates
```

**Example with Selection Parameter:**
```
User brushes region → brush captures selected points → opacity condition evaluates → visual updates
```
---

## Section B: Selection Parameters 

### Conceptual Foundation

**Selection parameters capture user interactions with the visualization itself.** Unlike variable parameters that work with external widgets, selections respond to mouse events, touch gestures, and keyboard input directly on the chart.

Key concepts:
- **Selections as data queries**: Each selection defines which data points meet certain criteria
- **Event types**: Different interactions (click, hover, brush) serve different purposes
- **Selection vs Response**: Capturing the interaction is separate from responding to it


### Selection Point Patterns

`alt.selection_point()` captures discrete data points through user interaction.

| **Pattern** | **Code** | **User Interaction** | **Use Case** |
|---|---|---|---|
| **Basic Click** | `alt.selection_point()` | Click to select, click again to deselect | Highlighting individual data points |
| **Hover** | `alt.selection_point(on='pointerover')` | Mouse over to select | Immediate visual feedback without clicking |
| **Nearest Point** | `alt.selection_point(nearest=True, on='pointerover')` | Hover near any point to select closest one | Precise selection in dense scatter plots |
| **Multi-Select** | `alt.selection_point(toggle='toggle')` | Shift+click to add/remove from selection | Comparing multiple individual points |
| **Field-Based** | `alt.selection_point(fields=['species'])` | Click one point to select all with same field value | Selecting entire categories at once |
| **Legend Binding** | `alt.selection_point(fields=['species'], bind='legend')` | Click legend items to filter | Interactive legend controls |

**Example: Basic Click vs Hover vs Nearest**

In [ ]:
cars = data.cars()

# Pattern 1: Basic Click
click_select = alt.selection_point()

chart1 = alt.Chart(cars).add_params(
    click_select
).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    opacity=alt.when(click_select).then(alt.value(1.0)).otherwise(alt.value(0.3))
).properties(title="Click to Select")

# Pattern 2: Hover
hover_select = alt.selection_point(on='pointerover')

chart2 = alt.Chart(cars).add_params(
    hover_select
).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    size=alt.when(hover_select).then(alt.value(150)).otherwise(alt.value(60))
).properties(title="Hover to Select")


# Pattern 3: Nearest Point (most useful for dense plots)
nearest_select = alt.selection_point(
    nearest=True, 
    on='pointerover',
    empty=False  # Always keep something selected
)

chart3 = alt.Chart(cars).add_params(
    nearest_select
).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    stroke=alt.when(nearest_select).then(alt.value('black')),
    strokeWidth=alt.when(nearest_select).then(alt.value(3)).otherwise(alt.value(0))
).properties(title="Nearest Point on Hover")


# Show all three
chart1 | chart2 | chart3

#### Understanding Empty Selection Behavior

By default, when nothing is selected (`empty=True`), the conditional encoding doesn't apply. Notice that when we change it to `False` for chart3, how there is ALWAYS a mark selected

#### Pulse Check: Question 1

This code is supposed to create a scatter plot where clicking a point highlights all points from the same origin, but it's not working. 
Find and fix the two errors:

In [ ]:
# Student wants: Click any Japanese car → all Japanese cars highlight
# What actually happens: Nothing

origin_click = alt.selection_point()

chart = alt.Chart(cars).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.when(origin_click).then('Origin:N').otherwise(alt.value('gray'))
).properties(
    title="Click a point to highlight its entire origin group",
    width=400,
    height=300
)

chart

#### Pulse Check: Question 2

You want to select points by hovering and you want the selected points to be large (size=200) and unselected points to be small (size=50). Your code throws an error or doesn't work as expected.
There are three errors. Identify and fix them. 

In [ ]:
hover_select = alt.selection_point()

chart = alt.Chart(cars).add_params(
    hover_select
).mark_circle().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    size=alt.when(hover_select).then(200).otherwise(50)
).properties(
    title="Hover to see larger points",
    width=400,
    height=300
)

chart


### Selection Interval Patterns

`alt.selection_interval()` captures rectangular regions through click-and-drag interactions.

| **Pattern** | **Code** | **User Interaction** | **Use Case** |
|---|---|---|---|
| **Basic Brush** | `alt.selection_interval()` | Click and drag to select rectangle (both x and y) | General range selection |
| **X-Axis Only** | `alt.selection_interval(encodings=['x'])` | Drag horizontally to select x-range | Time series filtering, horizontal range selection |
| **Y-Axis Only** | `alt.selection_interval(encodings=['y'])` | Drag vertically to select y-range | Value range filtering, vertical range selection |
| **Preset Selection** | `alt.selection_interval(init={'x': [80, 120], 'y': [20, 30]})` | Chart loads with initial selection | Guided exploration with default focus region |

**Example: Basic Brush vs X-Only vs Y-Only**


In [ ]:
# Pattern 1: Basic Brush (both axes)
brush = alt.selection_interval()

chart1 = alt.Chart(cars).add_params(
    brush
).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.when(brush).then('Origin:N').otherwise(alt.value('lightgray'))
).properties(title="Brush Both Axes", width = 250, height = 250)

# Pattern 2: X-Axis Only
x_brush = alt.selection_interval(encodings=['x'])

chart2 = alt.Chart(cars).add_params(
    x_brush
).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.when(x_brush).then('Origin:N').otherwise(alt.value('lightgray'))
).properties(title="Brush X-Axis Only", width = 250, height = 250)

# Pattern 3: Y-Axis Only
y_brush = alt.selection_interval(encodings=['y'])

chart3 = alt.Chart(cars).add_params(
    y_brush
).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.when(y_brush).then('Origin:N').otherwise(alt.value('lightgray'))
).properties(title="Brush Y-Axis Only", width = 250, height = 250)

# Show all three
chart1 | chart2 | chart3


#### Build Coordinated Scatter Plots


In [ ]:
# Create a single brush selection that will be shared across both charts
###brush = 

# Scatter Plot 1: Performance view (Horsepower vs MPG)
# This is where users make the brush selection
performance_plot = alt.Chart(cars).mark_circle(size=60).encode(
    x=alt.X('Horsepower:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Miles_per_Gallon:Q', scale=alt.Scale(zero=False)),
    #color=...
    #opacity=...
    tooltip=['Name:N', 'Origin:N', 'Horsepower:Q', 'Miles_per_Gallon:Q', 'Weight_in_lbs:Q']
).properties(
    width=350,
    height=300,
    title='Performance: Brush to select vehicles'
)

# Scatter Plot 2: Physical characteristics view (Weight vs Acceleration)
# This chart shows the SAME selection, different dimensions
physical_plot = alt.Chart(cars).mark_circle(size=60).encode(
    x=alt.X('Weight_in_lbs:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Acceleration:Q', scale=alt.Scale(zero=False)),
  #  color=...
  #  opacity=...
    tooltip=['Name:N', 'Origin:N', 'Weight_in_lbs:Q', 'Acceleration:Q']
).properties(
    width=350,
    height=300,
    title='Physical: See selected vehicles here'
)

# Display side by side
performance_plot | physical_plot

### **Scenario**

You're analyzing car performance data and want to understand relationships across different dimensions. You have two scatter plots showing different aspects of the cars:
- **Left plot:** Performance (Horsepower vs Miles per Gallon)  
- **Right plot:** Physical characteristics (Weight vs Acceleration)

Your goal: **Use a brush selection in the left plot to highlight the same cars in the right plot**, revealing how performance relates to physical characteristics.

---

### **Your Task**

Complete the code below to create coordinated highlighting across both scatter plots. The brush selection has been created for you, but you need to:

1. **Define the brush**
2. **Register the brush** with the appropriate chart
3. **Add conditional color encoding** to both plots (selected points show `Origin:N`, unselected points are `'lightgray'`)
4. **Add conditional opacity encoding** to both plots (selected points have opacity `1.0`, unselected points have opacity `0.3`)

---

### **Starter Code**

```python

# Create a single brush selection that will be shared across both charts
brush = ...

# Scatter Plot 1: Performance view (Horsepower vs MPG)
# This is where users make the brush selection
performance_plot = alt.Chart(cars).mark_circle(size=60).encode(
    x=alt.X('Horsepower:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Miles_per_Gallon:Q', scale=alt.Scale(zero=False)),
    color=...,  # TODO: Add conditional color encoding
    opacity=...,  # TODO: Add conditional opacity encoding
    tooltip=['Name:N', 'Origin:N', 'Horsepower:Q', 'Miles_per_Gallon:Q', 'Weight_in_lbs:Q']
).properties(
    width=350,
    height=300,
    title='Performance: Brush to select vehicles'
)

# Scatter Plot 2: Physical characteristics view (Weight vs Acceleration)
# This chart shows the SAME selection, different dimensions
physical_plot = alt.Chart(cars).mark_circle(size=60).encode(
    x=alt.X('Weight_in_lbs:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Acceleration:Q', scale=alt.Scale(zero=False)),
    color=...,  # TODO: Add conditional color encoding
    opacity=...,  # TODO: Add conditional opacity encoding
    tooltip=['Name:N', 'Origin:N', 'Weight_in_lbs:Q', 'Acceleration:Q']
).properties(
    width=350,
    height=300,
    title='Physical: See selected vehicles here'
)

# Display side by side
performance_plot | physical_plot

```

---

### **Step-by-Step Instructions**

#### **Step 1: Define & Register the brush (1 minute)**

The brush selection needs to be registered with **one** of the charts. Which chart should it be?

**Hint:** Users need to be able to drag and create the brush selection somewhere. Where do you want them to interact?
    
---

#### **Step 2: Add conditional color encoding (3 minutes)**

Both scatter plots need conditional color encoding:
- **When points are selected (inside the brush):** Color by `'Origin:N'`
- **When points are NOT selected (outside the brush):** Color should be `'lightgray'`

**Hint:** Use `alt.when(brush).then(...).otherwise(...)`

**Remember:** 
- Data field references like `'Origin:N'` don't need `alt.value()`
- Literal values like `'lightgray'` need `alt.value()`

Replace both `color=...` lines with your conditional encoding.

---

#### **Step 3: Add conditional opacity encoding (3 minutes)**

Both scatter plots need conditional opacity encoding:
- **When points are selected:** Opacity should be `1.0` (fully opaque)
- **When points are NOT selected:** Opacity should be `0.3` (faded)

**Hint:** Use the same `alt.when(brush).then(...).otherwise(...)` pattern

Replace both `opacity=...` lines with your conditional encoding.

---

#### **Step 4: Test your visualization (2 minutes)**

Run your code and test:
1. Can you create a brush selection by clicking and dragging in the left plot?
2. Do the selected points in the left plot get colored by Origin?
3. Do the selected points in the left plot become fully opaque?
4. Do the **same points** get highlighted in the right plot?
5. Can you answer: "Do high-MPG cars tend to be lighter?"

---

### **Expected Behavior**

When working correctly:
- Brushing in the **Performance plot** highlights points in **both** plots
- Selected points are colored by Origin (USA, Europe, Japan)
- Selected points are fully visible (opacity 1.0)
- Unselected points are gray and faded (opacity 0.3)
- The same cars are highlighted in both views, letting you explore multi-dimensional relationships

---

### **Reflection Questions** (3 minutes)

After completing the activity, discuss with a partner:

1. **Why do we only add `.add_params(brush)` to one chart, not both?**

2. **What insights can you discover using these coordinated views?** Try brushing:
   - High horsepower, low MPG cars → What do you notice about their weight?
   - Low horsepower, high MPG cars → How do they accelerate?

3. **Could you add a third coordinated view?** What other car attributes would be interesting to explore?

4. **What would happen if you used `transform_filter(brush)` instead of conditional encoding?** How would the behavior differ?



## Using Selections to Filter Data

### Core Concept: Selection as Predicate

When you use a selection parameter in `transform_filter()`, it acts as a **predicate** (a true/false test):
- Data points **inside** the selection evaluate to `True` → **kept**
- Data points **outside** the selection evaluate to `False` → **filtered out**

**Key Rule:** You cannot both select and filter in the same chart. Filtering requires **at least two charts** - one to make the selection, another to show the filtered result.

---


### Pattern 1: Brush Selection → Filter
**Use Case:** Explore data ranges in one view, see filtered results in another


In [ ]:

# Create brush selection
brush = alt.selection_interval()

# Chart 1: Make selection here
scatter = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.when(brush).then('Origin:N').otherwise(alt.value('gray'))
).add_params(
    brush
).properties(
    width=350,
    height=250,
    title="Brush to select data range"
)

# Chart 2: Show filtered results
bars = alt.Chart(cars).mark_bar().encode(
    x='count():Q',
    y='Origin:N',
    color='Origin:N'
).transform_filter(
    brush  # Filter based on brush selection
).properties(
    width=200,
    height=100,
    title="Counts update based on selection"
)

scatter | bars



**What's happening:**
1. User drags to create selection in scatter plot
2. `brush` captures which points are selected, points updated to show which ones are selected
3. `transform_filter(brush)` keeps only selected data
4. Bar chart updates to show counts of selected data only

---

### Pattern 2: Point Selection → Filter

**Use Case:** Click categories in one view to filter details in another


In [ ]:
# Create point selection on a categorical field
origin_select = alt.selection_point(fields=['Origin'])

# Chart 1: Click to select origin
origin_bars = alt.Chart(cars).mark_bar().encode(
    x='count():Q',
    y='Origin:N',
    color=alt.when(origin_select).then('Origin:N').otherwise(alt.value('lightgray')),
    opacity=alt.when(origin_select).then(alt.value(1.0)).otherwise(alt.value(0.5))
).add_params(
    origin_select
).properties(
    width=200,
    height=100,
    title="Click to select origin"
)

# Chart 2: Show only selected origin's data
scatter_filtered = alt.Chart(cars).mark_circle(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    tooltip=['Name:N', 'Origin:N', 'Horsepower:Q', 'Miles_per_Gallon:Q']
).transform_filter(
    origin_select  # Filter based on origin selection
).properties(
    width=300,
    height=220,
    title="Details for selected origin"
)

origin_bars | scatter_filtered


**Key difference from brush:** 
- Field-based selection (`fields=['Origin']`) selects by category
- Clicking USA bar filters to show only USA cars in scatter plot
- Notice how the axes values change
---

### Pattern 3: Multiple Linked Views

**Use Case:** Coordinated filtering across multiple analytical views


In [ ]:
# Create brush for continuous selection
brush = alt.selection_interval()

# Overview: Select by horsepower and MPG
overview = alt.Chart(cars).mark_point(size=30).encode(
    y='Horsepower:Q',
    x='Miles_per_Gallon:Q',
    color=alt.when(brush).then('Origin:N').otherwise(alt.value('green')),
    tooltip=['Name:N']
).add_params(
    brush
).properties(
    width=200,
    height=600,
    title="1. Select vehicles by performance"
)

# Detail 1: Distribution of years for selected vehicles
year_hist = alt.Chart(cars).mark_bar().encode(
    x=alt.X('year(Year):O', title='Year'),
    y='count():Q',
    color='Origin:N'
).transform_filter(
    brush
).properties(
    width=300,
    height=160,
    title="2. WHEN and WHERE were they made?"
)

# Detail 2: Relationship between Acceleration and Displacement
dis_acc_scatter = alt.layer(alt.Chart(cars).mark_circle(fill='black').encode(
    x='Displacement:Q',
    y='Acceleration:Q',
).transform_filter(brush),
alt.Chart(cars).transform_filter(
        brush
    ).transform_regression(
        'Displacement', 'Acceleration'
    ).mark_line(color='purple', size=3).encode(
    x='Displacement:Q',
    y='Acceleration:Q'
)).properties(width=300, height=220, title="3. WHAT is the relationship?")

# Detail 3: Average weight for selected vehicles
weight_bar = alt.Chart(cars).mark_bar().encode(
    x='average(Weight_in_lbs):Q',
    y='Origin:N',
    color='Origin:N'
).transform_filter(
    brush
).properties(
    width=300,
    height=90,
    title="4. HOW heavy are they?"
)




# Layout: overview on left, details on right
overview | (year_hist &  dis_acc_scatter & weight_bar)


### Key Takeaways

1. **Filtering requires two charts**: One to make the selection, another to show filtered results
2. **Brush selections** are perfect for continuous range filtering
3. **Field-based selections** are ideal for categorical filtering
4. **Multiple views can share one selection** for coordinated analysis
5. **Choose filter vs. conditional encoding** based on whether you want to remove or dim data

### Quick Reference: Choosing the Right Selection

| **If you want to...** | **Use this** |
|---|---|
| Let users click individual points | `alt.selection_point()` |
| Show instant feedback on hover | `alt.selection_point(on='pointerover')` |
| Select nearest point in dense data | `alt.selection_point(nearest=True, on='pointerover')` |
| Select entire categories at once | `alt.selection_point(fields=['category_column'])` |
| Let users drag to select ranges | `alt.selection_interval()` |
| Filter a time series by date range | `alt.selection_interval(encodings=['x'])` |
| Filter by value ranges vertically | `alt.selection_interval(encodings=['y'])` |

## Get Stepping
We have just scratched the surface.
We have provided you with a foundation, but notice how we didn't spend any time today on variables. Go online and learn that on your own. 
In addition, start taking the interactive visualizations that exist on Altair's website and start deconstructing them and making new things. 
This is our last in class programming lecture. 
Now it is up to you to explore, break things, fix them and create new things. 


There will be two more tutorials posted next week for Color and Map. These you will do entirely on your own. No inclass time on them. 